## 准备数据

In [6]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [7]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [8]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
         # 初始化权重和偏置
        # 假设输入大小为28*28=784，输出大小为10（数字0-9）
        # 隐藏层大小可以自定义，这里我们设置为128
        self.W1 = tf.Variable(tf.random.normal([784, 128], stddev=0.03), name='W1')
        self.b1 = tf.Variable(tf.random.normal([128]), name='b1')
        
        self.W2 = tf.Variable(tf.random.normal([128, 64], stddev=0.03), name='W2')
        self.b2 = tf.Variable(tf.random.normal([64]), name='b2')
        
        self.W3 = tf.Variable(tf.random.normal([64, 10], stddev=0.03), name='W3')
        self.b3 = tf.Variable(tf.random.normal([10]), name='b3')
        ####################
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        # 前向传播过程
        # 输入x形状为(batch_size, 28, 28)，需要先展平成(batch_size, 784)
        x = tf.reshape(x, (-1, 784))
        
        # 第一层
        z1 = tf.add(tf.matmul(x, self.W1), self.b1)
        a1 = tf.nn.relu(z1)
        
        # 第二层
        z2 = tf.add(tf.matmul(a1, self.W2), self.b2)
        a2 = tf.nn.relu(z2)
        
        # 输出层
        logits = tf.add(tf.matmul(a2, self.W3), self.b3)
        ####################
        return logits
        
model = myModel()

optimizer = optimizers.Adam()


## 计算 loss

In [11]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # 计算梯度
    trainable_vars = [model.W1, model.W2, model.W3, model.b1, model.b2, model.b3]
    grads = tape.gradient(loss, trainable_vars)
    
    # 使用优化器更新权重
    optimizer.apply_gradients(zip(grads, trainable_vars))

    accuracy = compute_accuracy(logits, y)

    # 返回损失和准确率
    return loss, accuracy
@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [13]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 0.8226924 ; accuracy 0.7658167
epoch 1 : loss 0.79667693 ; accuracy 0.77315
epoch 2 : loss 0.77210164 ; accuracy 0.78085
epoch 3 : loss 0.74881727 ; accuracy 0.78746665
epoch 4 : loss 0.72680545 ; accuracy 0.7948667
epoch 5 : loss 0.7060169 ; accuracy 0.8013167
epoch 6 : loss 0.686283 ; accuracy 0.80646664
epoch 7 : loss 0.66745174 ; accuracy 0.81093335
epoch 8 : loss 0.64953023 ; accuracy 0.8146833
epoch 9 : loss 0.632575 ; accuracy 0.8185167
epoch 10 : loss 0.6165531 ; accuracy 0.8224
epoch 11 : loss 0.6013944 ; accuracy 0.82591665
epoch 12 : loss 0.5870521 ; accuracy 0.82998335
epoch 13 : loss 0.57344806 ; accuracy 0.83353335
epoch 14 : loss 0.56048113 ; accuracy 0.83715
epoch 15 : loss 0.54814553 ; accuracy 0.8401833
epoch 16 : loss 0.5364797 ; accuracy 0.84318334
epoch 17 : loss 0.5254328 ; accuracy 0.84601665
epoch 18 : loss 0.51491296 ; accuracy 0.84886664
epoch 19 : loss 0.5048901 ; accuracy 0.8523
epoch 20 : loss 0.4953445 ; accuracy 0.8548
epoch 21 : loss 0.486